In [ ]:
# redis 测试
import redis
from wxpy import *
from chatterbot import ChatBot
from chatterbot.trainers import ListTrainer
 
# Create a new chat bot named Charlie
chatbot = ChatBot('联通机器人')
 
trainer = ListTrainer(chatbot)
 
trainer.train([
    "联通","我爱联通公司","你呢"
])
 

# init redis
r = redis.StrictRedis(host='localhost', port=6379, db=0,decode_responses=True)
# init bot
bot = Bot()

# info store in redis as list
def group_info_store(info):
    r.rpush('group_info',info)
# export group info
def group_info_export():
    size=r.llen("group_info") 
    for i in range(0,size):
        print(r.lindex("group_info",i))
# delete group info
def remove_group_info():
    size=r.llen("group_info") 
    for i in range(0,size):
        r.brpop('group_info')

# deal info with group   
@bot.register(Group,TEXT)
def auto_accept_friends(msg):
#     msg.chat.send(msg.text)
    username=msg.member.name
    info=msg.text
    user_info=str(username)+":"+str(info)
    print(info)
    print(chatbot.get_response(info))
    msg.chat.send(chatbot.get_response(info))
    group_info_store(user_info)
    
    if(info=='导出聊天记录'):
        print('----导出中----')
        group_info_export()
        print('----导出完成----')
    elif(info=='删除聊天记录'):
        print('----删除中----')
        remove_group_info()
        print('----删除完成----')
        msg.chat.send('删除完成')
@bot.register(Friend,TEXT)
def auto_accept_friends(msg):
    msg.chat.send(msg.text)


In [81]:
# chatterbot 机器人回答
import redis
r = redis.StrictRedis(host='localhost', port=6379, db=0,decode_responses=True)
from chatterbot import ChatBot
from chatterbot.trainers import ListTrainer
# Create a new chat bot named 联通机器人
chatbot = ChatBot('联通机器人')
# init robot
trainer = ListTrainer(chatbot)


# add train data
def insert_data(info):
    r.rpush('data',info)
    train_data()
# get all data
def get_all_data():
    size=r.llen("data")
    train_data=[]
    for i in range(0,size):
        train_data.append(r.lindex("data",i))
    return train_data
    
# use robot train data
def train_data():
    data=get_all_data()
    trainer.train(data)
    return trainer


In [1]:
# import gensim
# import jieba
# import numpy as np
# from scipy.linalg import norm
# file_path= 'news_12g_baidubaike_20g_novel_90g_embedding_64.bin'
# model = gensim.models.KeyedVectors.load_word2vec_format(file_path, binary=True)
# def vector_similarity(s1, s2):
#     def sentence_vector(s):
#         words = jieba.lcut(s)
#         v = np.zeros(64)
#         for word in words:
#             v += model[word]
#         v /= len(words)
#         return v
#     v1, v2 = sentence_vector(s1), sentence_vector(s2)
#     return np.dot(v1, v2) / (norm(v1) * norm(v2))

# print('模型加载完成')

模型加载完成


In [101]:
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
from scipy.linalg import norm


def tfidf_similarity(s1, s2):
    def add_space(s):
        return ' '.join(list(s))
    
    # 将字中间加入空格
    s1, s2 = add_space(s1), add_space(s2)
    # 转化为TF矩阵
    cv = TfidfVectorizer(tokenizer=lambda s: s.split())
    corpus = [s1, s2]
    vectors = cv.fit_transform(corpus).toarray()
    # 计算TF系数
    return np.dot(vectors[0], vectors[1]) / (norm(vectors[0]) * norm(vectors[1]))


In [102]:
def cal_diff_setence_2(info,redis_list):
    marks=dict()
    mark=list()
    for i in redis_list: 
        score=tfidf_similarity(info, i)
        mark.append(score)
        marks[i]=score
    if(max(mark)<0.45):
        return None
    else:
        return (max(marks, key=marks.get))

In [107]:
# map 匹配回答
import redis
from wxpy import *
import redis
# 初始化redis
r = redis.StrictRedis(host='localhost', port=6379, db=0,decode_responses=True)
# 初始化机器人
bot = Bot()
bot.enable_puid('wxpy_puid.pkl')
# 插入题库
# 插入方式，问题：你好，请问你叫什么？  答案：我叫张润民
def take_Q_S(info):
    try:
        first_index=pattern.search(info).span(0)[1]+1
        tempxx=(info[first_index:])
        second_index=tempxx.find('答')+first_index
        question=info[first_index:second_index].strip()
        answer=info[second_index+3:].strip()
        print(question,answer)
    except:
        print ("录入问题出错")
    return question,answer
# 数据插入map
def insert_map_data(question,answer):
    r.hset("QandA", question, answer)
    print('问题已经录入成功')
# 获取答案
def get_answer(question):
    redis_list_key=list(r.hgetall("QandA") .keys())
    answer=cal_diff_setence_2(question,redis_list_key)
    if answer==None:
        return None
    else:
        return r.hget("QandA", answer)
# 删除数据
# def delete_data(info):
#     r.hdel("QandA",info)
#     print('删除成功')

# info store in redis as list
def group_info_store(info):
    r.rpush('group_info',info)
# export group info
def group_info_export():
    size=r.llen("group_info") 
    for i in range(0,size):
        print(r.lindex("group_info",i))
# delete group info
def remove_group_info():
    size=r.llen("group_info") 
    for i in range(0,size):
        r.brpop('group_info')
# deal info with group   
@bot.register(Group,TEXT)
def auto_accept_friends(msg):
#     msg.chat.send(msg.text)
    info=msg.text
    if('导出聊天记录'not in msg.text):
        username=msg.member.name
    #     print(msg.member.puid)
    #     print(type(msg.member.puid))
        user_info=str(username)+":"+str(info)
        group_info_store(user_info)
        if(get_answer(info[:])==None):
            a='别问啦，我还需要学习呢'
        else:
            a=get_answer(info[:])
        if (info.strip()[0] in ['~','～']):
            msg.chat.send(a)
    #         print(bot.self.name)
            myself_name=bot.self.name
            myself_info=str(myself_name)+":"+str(a)
            group_info_store(myself_info)
            
        elif(info.strip()[0] in ['~','～']):
            msg.chat.send(a)
            myself_name=bot.self.name
            myself_info=str(myself_name)+":"+str(a)
            group_info_store(myself_info)
        else:
            raise Exception("系统出问题了")
    if (info.strip()[0] in ['~','～']) and '导出聊天记录' in info:
        
        print('-------------导出中---------------')
        group_info_export()
        print('-------------导出完成-------------')

Getting uuid of QR code.
Please scan the QR code to log in.
Please press confirm on your phone.
LOG OUT!
Loading the contact, this may take a little while.
Login successfully as 我是个无情的机器人


-------------导出中---------------
ZRM:11
ZRM:～1
ZRM:~导出聊天记录
ZRM:～～
我是个无情的机器人:别问啦，我还需要学习呢
ZRM:~导出聊天记录
我是个无情的机器人:别问啦，我还需要学习呢
Punchoo:～
我是个无情的机器人:别问啦，我还需要学习呢
萌萌:~
我是个无情的机器人:别问啦，我还需要学习呢
ZRM:@冯雷昊 @张文萱 @张润民 @吴宗翰 请各位组长辛苦一下，收集一下新员工电子照片，要白底的，请于明天下班前发给我。
ZRM:~你好
ZRM:~你好
沁心🌸☘:你好
付昊博:失败
ZRM:a
ZRM:～
付昊博: 那您吃啥
ZRM:~
沁心🌸☘:休息吧
ZRM:a
ZRM:~你好
ZRM:～你好好
ZRM:a
ZRM:~a
ZRM:a
沁心🌸☘:你可以休息了
沁心🌸☘:机器人
ZRM:a
ZRM:～你好
ZRM:～你好
ZRM:~你好
ZRM:你好
ZRM:你好
沁心🌸☘:jieba
沁心🌸☘:这个包不错
ZRM:nihao
ZRM:有了
沁心🌸☘:你也用了
ZRM:nihao
ZRM:你好
ZRM:你好
ZRM:你好
ZRM:nhao
ZRM:nihao
ZRM:你好
ZRM:～你好
我是个无情的机器人:你好
ZRM:～你会干嘛
我是个无情的机器人:我能帮你处理事情
ZRM:～你能做什么
我是个无情的机器人:我叫张润民
ZRM:～你叫什么名字
我是个无情的机器人:我叫张润民
ZRM:～你好
我是个无情的机器人:你好
ZRM:～你会干嘛
我是个无情的机器人:我能帮你处理事情
ZRM:～你会吃屎么
我是个无情的机器人:我能帮你处理事情
ZRM:～导出聊天记录
我是个无情的机器人:我叫张润民
Lovely Socre:[旺柴][旺柴]
ZRM:你好
ZRM:你好
ZRM:～你好
我是个无情的机器人:你好
ZRM:～你你那舒服的
我是个无情的机器人:别问啦，我还需要学习呢
ZRM:～你能做啥
我是个无情的机器人:别问啦，我还需要学习呢
ZRM:～你会干嘛
我是个无情的机器人:我能帮你处理事情
沁心🌸☘:你也太骚
沁心🌸☘:受不了
沁心🌸☘:下去吧
ZRM:前面加～
ZRM:是和机器人对话
ZRM:～你也太骚了
我是个无情的机器人:放屁，你说给文明点
ZRM:～你也太骚了
我是个无情的机器人:放屁，你说话文明点
